In [3]:
import math
import csv

In [41]:
def F(x,y,z):
  return (x & y) | (~x & z)

def G(x,y,z):
  return (x & z) | (y & ~z)

def H(x,y,z):
  return x ^ y ^ z

def I(x,y,z):
  return y ^ (x | ~z)

def left_rotate(x, amount):
    x &= 0xFFFFFFFF
    return (((x<<amount))| (x>>(32-amount))) & 0xFFFFFFFF

def md5_to_hex(digest):
    raw = digest.to_bytes(16, byteorder='little')
    return '{:032x}'.format(int.from_bytes(raw, byteorder='big'))

def custom_md5(m, h0):
  T = [ 0 for i in range(64)]
  a0 = (h0 // (2 ** (32 * 3))) % (2 ** 32)
  b0 = (h0 // (2 ** (32 * 2))) % (2 ** 32)
  c0 = (h0 // (2 ** 32)) % (2 ** 32)
  d0 = h0 % (2 ** 32)

  constantes = [a0,b0 ,c0 ,d0]
  message = bytearray(m, encoding="utf-8")
  leninbits = (8 * len(message)) & 0xffffffffffffffff
  message.append(0x80)
  while len(message)% 64 != 56:
    message.append(0)
  message += leninbits.to_bytes(8, byteorder="little")


  for i in range(0, 64):
    T[i] = int(abs(math.sin(i+1)) * 2**32) & 0xFFFFFFFF 

  a,b,c,d = constantes[:]
  for p in range(0, len(message) ,64):
    x = message[p:p+64]
    words = []
    for j in range(0,64,4):
      words.append(x[j:j+4])
    AA, BB, CC , DD = a,b,c,d
    i = 0

    #Round 1
    s = [7, 12, 17, 22]
    
    for l in range(4):
      a = (b + left_rotate(a + F(b,c,d) + T[i] + int.from_bytes(words[i%16], byteorder='little'),s[0])) & 0xFFFFFFFF
      i += 1
      d = (a + left_rotate(d + F(a,b,c) + T[i] + int.from_bytes(words[i%16], byteorder='little'),s[1])) & 0xFFFFFFFF
      i += 1
      c = (d + left_rotate(c + F(d,a,b) + T[i] + int.from_bytes(words[i%16], byteorder='little'),s[2])) & 0xFFFFFFFF
      i += 1
      b = (c + left_rotate(b + F(c,d,a) + T[i] + int.from_bytes(words[i%16], byteorder='little'),s[3])) & 0xFFFFFFFF
      i += 1


    #Round 2
    s = [5, 9, 14, 20]
    f = lambda i: (5*i + 1)%16
    for l in range(4):
      a = (b + left_rotate(a + G(b,c,d) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[0])) & 0xFFFFFFFF
      i += 1
      d = (a + left_rotate(d + G(a,b,c) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[1])) & 0xFFFFFFFF
      i += 1
      c = (d + left_rotate(c + G(d,a,b) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[2])) & 0xFFFFFFFF     
      i += 1
      b = (c + left_rotate(b + G(c,d,a) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[3])) & 0xFFFFFFFF      
      i += 1


    #Round 3
    s = [4, 11, 16, 23]
    f = lambda i: (3*i + 5)%16
    for l in range(4):
      a = (b + left_rotate(a + H(b,c,d) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[0])) & 0xFFFFFFFF
      i += 1
      d = (a + left_rotate(d + H(a,b,c) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[1])) & 0xFFFFFFFF
      i += 1
      c = (d + left_rotate(c + H(d,a,b) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[2])) & 0xFFFFFFFF
      i += 1
      b = (c + left_rotate(b + H(c,d,a) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[3])) & 0xFFFFFFFF
      i += 1

    #Round 4
    s = [6, 10, 15, 21]
    f = lambda i: (7*i)%16
    for l in range(4):
      a = (b + left_rotate(a + I(b,c,d) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[0])) & 0xFFFFFFFF
      i += 1
      d = (a + left_rotate(d + I(a,b,c) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[1])) & 0xFFFFFFFF
      i += 1
      c = (d + left_rotate(c + I(d,a,b) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[2])) & 0xFFFFFFFF
      i += 1
      b = (c + left_rotate(b + I(c,d,a) + T[i] + int.from_bytes(words[f(i%16)], byteorder='little'),s[3])) & 0xFFFFFFFF
      i += 1
    a += AA
    a &= 0xFFFFFFFF
    b += BB
    b &= 0xFFFFFFFF
    c += CC
    c &= 0xFFFFFFFF
    d += DD
    d &= 0xFFFFFFFF
  hash_pieces = [a,b,c,d]
  return md5_to_hex(sum(x<<(32*i) for i, x in enumerate(hash_pieces)))




In [117]:
encontre = True
i = 0
diccionario = dict()
with open('mensajes_p3.csv', newline='') as csvfile:
  messages = csv.reader(csvfile, delimiter=',', quotechar='"')
  for row in messages:
    diccionario[row[0]]= row[1]

encontro = False
messages = []
messages_int = []
space_letter = []
letter_letter = []
while encontre:
  lista = [] 
  encontro = False
  hash = custom_md5("jotramon@uc.cl",16638824*100+i)
  for key in diccionario.keys():
    if key == hash:
      encontro = True
      lista.append(diccionario[key])
  if not encontro:
    break
  palabra = []
  palabra_int = []
  for j in range(0,len(lista[0]),8):
    palabra.append(lista[0][j:j+8])
    palabra_int.append(int(lista[0][j:j+8], 2))
  messages.append(palabra)
  messages_int.append(palabra_int)


  i+=1
groups = []
clasificadas = []
for mes in range(len(messages_int)):
  if mes in clasificadas:
      continue
  grupo = []
  for mes_2 in range(len(messages_int)):
    if mes == mes_2:
      continue
    if mes_2 in clasificadas:
      continue
    count_1 = 0
    count_2 = 0
    for pos in range(10):
      if (messages_int[mes][pos]^messages_int[mes_2][pos] > 97):
        count_1 = 1
        break
      if (messages_int[mes][pos]^messages_int[mes_2][pos] < 32):
        count_2 +=1
      
    if count_1 == 0 and count_2 >3:
      clasificadas.append(mes_2)
      grupo.append(mes_2)
  groups.append(grupo)

probs = [ [0 for i in range(len(groups))] for j in messages ]
for i in range(3):
  for mes in range(len(messages_int)):
    for group in range(len(groups)):
      if len(groups[group]) == 0:
        continue
      favorables = 0
      for mes_2 in range(len(groups[group])):
        count_1 = 0
        count_2 = 0
        for pos in range(10):
          if (messages_int[mes][pos]^messages_int[groups[group][mes_2]][pos] > 97):
            count_1 = 1
          if (messages_int[mes][pos]^messages_int[groups[group][mes_2]][pos] < 32):
            count_2 += 1
        if count_1 == 0 and count_2 >1:
          favorables += 1
      probs[mes][group] = favorables/len(groups[group])
  new_groups = [[] for i in range(len(groups))]
  for mes in range(len(messages_int)):
    new_groups[probs[mes].index(max(probs[mes]))].append(mes)
  groups =new_groups

for group in range(len(groups)):
  if len( groups[group]) <15 and len( groups[group]) != 0 :
    for mes in groups[group]:
      probs[mes][probs[mes].index(max(probs[mes]))]= 0
      groups[probs[mes].index(max(probs[mes]))].append(mes)

def probability_space(word, words):
    length = len(word)
    count = [0] * length

    for i in words:
        for j in range(length):
          if word^words[i] >64:
            count[j] += 1
    return [round(i/len(words), 4) for i in count]

[probability_space()]

print(messages)
print(messages_int)
print(len(groups))
print(groups)


[['00011001', '01111101', '01100111', '01011001', '00001000', '00010011', '00100101', '00001110', '00001111', '00111000'], ['00011010', '01111101', '00110011', '01000001', '00001011', '00011010', '01100100', '00001101', '00011101', '01101010'], ['00000010', '01110011', '00101110', '01000100', '00010000', '01010111', '01100111', '00000110', '00001101', '00101011'], ['00000100', '01101011', '00100010', '00010110', '00010000', '00011111', '01100000', '00011010', '01001110', '00111001'], ['00010000', '01101111', '01100111', '01011110', '00000001', '00000101', '00100101', '00010100', '00011100', '00101011'], ['00000001', '01101000', '00101110', '01011000', '00000011', '01010111', '01110000', '00010011', '01001110', '00111001'], ['00010000', '01101101', '00110100', '01010111', '00000011', '00010010', '01110110', '01000011', '00000111', '00100100'], ['01010001', '01111001', '01100111', '01000110', '00001011', '00000100', '01110001', '00000110', '00011100', '01101010'], ['00011110', '01111110'